Handling Constraints
====================

A very common scenario that we face when working with tabular data is
finding columns that have very particular relationships between them
which are very hard to model and easily confuse the Tabular Models.

Some simple examples of these scenarios include:

-   A table that has the columns `country` and `city`: In such scenario,
    it might be very hard to learn which country each city belongs to,
    and when sampling probabilistically, the model is likely to end up
    generating invalid country/city combinations.
-   A table that contains both the `age` and the `date of birth` of a
    user. The model will learn the age and date of birth distributions
    and mostly generate valid combinations, but in some cases it might
    end up giving back ages that do not correspond to the given date of
    birth.

These kind of special relationships between columns are called
`Constraints`, and **SDV** provides a very powerful and flexible
mechanism to take them into account and guarantee that the sampled data
always respects them.

Let us explore a few `Constraint` examples and learn how to handle them:

Load a Tabular Demo
-------------------

We will start by loading a small table that contains data with some
constraints:

In [1]:
from sdv.demo import load_tabular_demo

employees = load_tabular_demo()
employees

,company,department,name,address,age,age_when_joined,years_in_the_company
0,Pear,Sales,Jessica Warren,"124 Hull Court Suite 765\nWest Michaelmouth, C...",42,42,0
1,Pear,Design,Margaret Cooley,"4124 Karen Street Suite 475\nWest Cherylbury, ...",47,44,3
2,Glasses,AI,Carlos Gibson,"7825 Watson View\nNew Alex, HI 06753",44,35,9
3,Glasses,Search Engine,Michelle Baker,"841 Chang Villages\nNorth Ryan, MS 46649",35,28,7
4,Cheerper,BigData,Colleen Stout,"72396 Sanders Isle\nCarlview, OR 22178",31,31,0
5,Cheerper,Support,Peter Williams,"552 Robinson Fall\nDeckerview, NM 59875",43,39,4
6,Pear,Sales,Kelly Warren,"4053 Garcia Causeway Apt. 468\nSouth Erica, TX...",38,35,3
7,Pear,Design,Carl Ruiz,"72635 Mary Spur Apt. 815\nRossville, MD 77009",30,28,2
8,Glasses,AI,Henry Schultz,"5607 Brittany Passage Apt. 698\nEast Matthew, ...",32,23,9
9,Glasses,Search Engine,Katherine Medina,Unit 0858 Box 8244\nDPO AP 37575,30,28,2


This step loaded a simple table that gives us some basic details about
simulated employees from several companies.

If we observe the data closely we will find a few **constraints**:

1.  Each `company` has employees from two or more `departments`, but
    `department` names are different across `companies`. This implies
    that a `company` should only be paired with its own `departments`
    and never with the `departments` of other `companies`.
2.  We have an `age` column that represents the age of the employee at
    the date when the data was created and an `age_when_joined` that
    represents the age of the employee when they joined the `company`.
    Since all of them joined the `company` before the data was created,
    the `age_when_joined` will always be equal or lower than the `age`
    column.
3.  We have a `years_in_the_company` column that indicates how many
    years passed since they joined the company, which means that the
    `years_in_the_company` will always be equal to the `age` minus the
    `age_when_joined`.

How does SDV Handle Constraints?
--------------------------------

**SDV** handles constraints using two different strategies:

### Transform Strategy

When using this strategy, **SDV** applies a transformation to the data
before learning it in a way that allows the model to better capture the
data properties. For example, if we have one column that needs to be
always greater than the other one, SDV can do the following:

1.  Replace the higher column with the difference between the two
    columns, which will always be positive.
2.  Model the transformed data and sample new values.
3.  Recompute the value of the high column by adding the values of the
    lower column to it.

The **Transform** strategy is very efficient and does not affect the
speed of the modeling and sampling process, but in some cases might
affect the quality of the learning process or simply not be possible.

### Reject Sampling Strategy

In the cases where applying a **Transform** strategy is not possible or
may affect the quality of the learning process, **SDV** can apply a
**Reject Sampling** strategy.

When using this strategy, **SDV** validates the sampled rows, discards
the ones that do not adjust to the constraint, and re-samples them. This
process is repeated until enough rows have been sampled.

Defining Constraints
--------------------

Let us go back to the demo data that we loaded before and define
**Constraints** that indicate **SDV** how to work with this data.

### FixedCombinations Constraint

The first constraint that we will explore is the `FixedCombinations`
constraint.

This Constraint class can handle the situation number 1 indicated above,
in which the values of a set of columns can only be combined exactly as
seen in the original data, and new combinations are not accepted. In
order to use this constraint we will need to import it from the
`sdv.constraints` module and create an instance of it indicating:

-   the names of the affected columns

In [2]:
from sdv.constraints import FixedCombinations

fixed_company_department_constraint = FixedCombinations(column_names=['company', 'department'])

### Inequality and ScalarInequality Constraints

Another constraint that we need for our data is the ``Inequality``
constraint. This constraint guarantees that one column is always greater
than the other one. In order to use it, we need to create an instance
passing:

-  the ``low_column_name``
-  the ``high_column_name``

In [3]:
from sdv.constraints import Inequality

age_gt_age_when_joined_constraint = Inequality(
    low_column_name='age_when_joined',
    high_column_name='age'
)

Using the Constraints
---------------------

Now that we have defined the constraints needed to properly describe our
dataset, we can pass them to the Tabular Model of our choice. For
example, let us create a `GaussianCopula` model passing it the
constraints that we just defined as a `list`:

In [6]:
from sdv.tabular import GaussianCopula

constraints = [
    fixed_company_department_constraint,
    age_gt_age_when_joined_constraint
]

gc = GaussianCopula(constraints=constraints)

After creating the model, we can just fit and sample as usual:

In [7]:
gc.fit(employees)

sampled = gc.sample(10)

And observe that the sampled rows really adjust to the constraints that
we defined:

In [8]:
sampled

,company,department,name,address,age,age_when_joined,years_in_the_company
0,Glasses,Search Engine,Carlos Gibson,"7825 Watson View\nNew Alex, HI 06753",38,38,0
1,Glasses,Search Engine,Jonathan Woodward,"11459 Tiffany Port\nLake Meganfort, OR 02415",46,41,5
2,Cheerper,BigData,Jonathan Woodward,"11459 Tiffany Port\nLake Meganfort, OR 02415",47,40,7
3,Glasses,AI,Henry Schultz,"5607 Brittany Passage Apt. 698\nEast Matthew, ...",33,23,10
4,Cheerper,BigData,Peter Williams,"552 Robinson Fall\nDeckerview, NM 59875",36,35,1
5,Pear,Design,Margaret Cooley,"4124 Karen Street Suite 475\nWest Cherylbury, ...",42,42,0
6,Pear,Sales,Kelly Warren,"4053 Garcia Causeway Apt. 468\nSouth Erica, TX...",33,29,4
8,Cheerper,BigData,Colleen Stout,"72396 Sanders Isle\nCarlview, OR 22178",41,33,8
0,Pear,Design,Jonathan Woodward,"11459 Tiffany Port\nLake Meganfort, OR 02415",40,30,10
1,Pear,Sales,Kelly Warren,"4053 Garcia Causeway Apt. 468\nSouth Erica, TX...",30,28,2
